In [1]:
# Example NumPy histogram data
import numpy as np
num_values = 10000
data = np.random.normal(loc=0, scale=1, size=num_values)


# Generate random weights
weights = np.random.uniform(0.5, 2.0, size=num_values)
bin_edges = np.linspace(-5, 5, 21)  # 20 bins between -5 and 5
hist_values, bin_edges = np.histogram(data, bins=bin_edges, weights=weights)
hist_w2, bin_edges = np.histogram(data, bins=bin_edges, weights=weights*weights)
centers = (bin_edges[:-1] + bin_edges[1:]) / 2.0

In [2]:
centers

array([-4.75, -4.25, -3.75, -3.25, -2.75, -2.25, -1.75, -1.25, -0.75,
       -0.25,  0.25,  0.75,  1.25,  1.75,  2.25,  2.75,  3.25,  3.75,
        4.25,  4.75])

In [3]:
from uproot3_methods.classes.TH1 import from_numpy
import uproot3

In [4]:
templates = []
th1 = from_numpy([hist_values, bin_edges])
th1._fName = "test"
th1._fSumw2 = np.array(np.append([0], hist_w2)) # -> np.array([0, group_sumw2])
th1._fTsumw2 = np.array(hist_w2).sum()
th1._fTsumwx2 = np.array(hist_w2 * centers).sum() # -> this may be wrong
templates.append(th1)
th1

<b'TH1D' 'test' 0x7f5c629c2ac0>

In [5]:
out_name = "test_dmitry_template.root"
out_file = uproot3.recreate(out_name)
for tmp in templates:
    out_file[tmp._fName] = tmp
out_file.close()

In [6]:
"""
Now use my own method and save that
"""
import ROOT


ERROR in cling::CIFactory::createCI(): cannot extract standard library include paths!
Invoking:
  LC_ALL=C x86_64-conda-linux-gnu-c++  -O3 -DNDEBUG -xc++ -E -v /dev/null 2>&1 | sed -n -e '/^.include/,${' -e '/^ \/.*++/p' -e '}'
Results was:
With exit code 0
C system headers (glibc/Xcode/Windows SDK) must be installed.
/depot/cms/kernels/python3/etc/cling/std.modulemap:69:12: error: header 'compare' not found
    header "compare"
           ^
input_line_1:1:10: note: submodule of top-level module 'std' implicitly imported here
#include <new>
         ^
Warning in cling::IncrementalParser::CheckABICompatibility():
  Failed to extract C++ standard library version.
Replaced symbol at_quick_exit cannot be found in JIT!
In file included from input_line_4:36:
In file included from /../lib/gcc/x86_64-redhat-linux/8/../../../../include/c++/8/cassert:43:
/../lib/gcc/x86_64-redhat-linux/8/../../../../include/c++/8/x86_64-redhat-linux/bits/c++config.h:3:10: fatal error: 'bits/wordsize.h' file not 

In [7]:
# Create a ROOT.TH1D histogram
hist_name = "test"
n_bins = len(hist_values)
hist = ROOT.TH1D(hist_name, "", n_bins, bin_edges[0], bin_edges[-1])

# Fill the ROOT.TH1D histogram with the bin contents
for i, value in enumerate(hist_values):
    hist.SetBinContent(i + 1, value)  # Bin index starts at 1 in ROOT

# Enable Sumw2 and overwrite it
hist.Sumw2()  # Ensure Sumw2 is enabled
sumw2_array = hist.GetSumw2()

# Overwrite the Sumw2 array values
# Overwrite Sumw2 with weights_squared
for i, w2 in enumerate(hist_w2, start=1):  # Start at 1 for ROOT bin indexing
    # print(f"w2 {w2}, for idx {i}")
    sumw2_array[i] = w2

# Verify the new Sumw2 values
print("Updated Sumw2 values:")
for i in range(0, n_bins + 1):
    print(f"Bin {i}, Sumw2: {sumw2_array[i]}")
print("Updated hist values:")
for i in range(0, n_bins + 1):
    print(f"Bin {i}, content: {hist.GetBinContent(i)}")

# Save the histogram to a ROOT file
output_file = ROOT.TFile("example_histogram.root", "RECREATE")
hist.Write()
output_file.Close()


Updated Sumw2 values:
Bin 0, Sumw2: 0.0
Bin 1, Sumw2: 0.0
Bin 2, Sumw2: 0.0
Bin 3, Sumw2: 0.8059265403285771
Bin 4, Sumw2: 22.036618690195592
Bin 5, Sumw2: 86.62772008836879
Bin 6, Sumw2: 260.22895042931395
Bin 7, Sumw2: 704.5782377952199
Bin 8, Sumw2: 1703.8480139043486
Bin 9, Sumw2: 2515.534792378889
Bin 10, Sumw2: 3305.771114513328
Bin 11, Sumw2: 3351.511468472143
Bin 12, Sumw2: 2618.027370742826
Bin 13, Sumw2: 1648.869059613031
Bin 14, Sumw2: 788.350454802774
Bin 15, Sumw2: 355.6576104226806
Bin 16, Sumw2: 85.65046269915183
Bin 17, Sumw2: 11.069030215014209
Bin 18, Sumw2: 1.2918794519755465
Bin 19, Sumw2: 3.0084339731220098
Bin 20, Sumw2: 0.0
Updated hist values:
Bin 0, content: 0.0
Bin 1, content: 0.0
Bin 2, content: 0.0
Bin 3, content: 1.2667087717563983
Bin 4, content: 16.005532081553838
Bin 5, content: 60.88478907222985
Bin 6, content: 189.83087158421444
Bin 7, content: 505.5257735503589
Bin 8, content: 1214.508430465436
Bin 9, content: 1797.5345379580265
Bin 10, content: 2380.

IncrementalExecutor::executeFunction: symbol '_ZN5cling7runtime6gClingE' unresolved while linking [cling interface function]!
You are probably missing the definition of cling::runtime::gCling
Maybe you need to load the corresponding shared library?
IncrementalExecutor::executeFunction: symbol '_ZN5cling7runtime6gClingE' unresolved while linking [cling interface function]!
You are probably missing the definition of cling::runtime::gCling
Maybe you need to load the corresponding shared library?
IncrementalExecutor::executeFunction: symbol '_ZN5cling7runtime6gClingE' unresolved while linking [cling interface function]!
You are probably missing the definition of cling::runtime::gCling
Maybe you need to load the corresponding shared library?
IncrementalExecutor::executeFunction: symbol '_ZN5cling7runtime6gClingE' unresolved while linking [cling interface function]!
You are probably missing the definition of cling::runtime::gCling
Maybe you need to load the corresponding shared library?
Incr

In [8]:
hist_w2
# hist_values

array([0.00000000e+00, 0.00000000e+00, 8.05926540e-01, 2.20366187e+01,
       8.66277201e+01, 2.60228950e+02, 7.04578238e+02, 1.70384801e+03,
       2.51553479e+03, 3.30577111e+03, 3.35151147e+03, 2.61802737e+03,
       1.64886906e+03, 7.88350455e+02, 3.55657610e+02, 8.56504627e+01,
       1.10690302e+01, 1.29187945e+00, 3.00843397e+00, 0.00000000e+00])

In [9]:
np.append([0], hist_w2)

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 8.05926540e-01,
       2.20366187e+01, 8.66277201e+01, 2.60228950e+02, 7.04578238e+02,
       1.70384801e+03, 2.51553479e+03, 3.30577111e+03, 3.35151147e+03,
       2.61802737e+03, 1.64886906e+03, 7.88350455e+02, 3.55657610e+02,
       8.56504627e+01, 1.10690302e+01, 1.29187945e+00, 3.00843397e+00,
       0.00000000e+00])

In [10]:
def compare_histograms(file1, hist_name1, file2, hist_name2):
    """
    Compare two TH1D histograms from different ROOT files.

    Args:
        file1 (str): Path to the first ROOT file.
        hist_name1 (str): Name of the histogram in the first file.
        file2 (str): Path to the second ROOT file.
        hist_name2 (str): Name of the histogram in the second file.

    Returns:
        bool: True if the histograms are identical, False otherwise.
    """
    # Open the ROOT files
    f1 = ROOT.TFile.Open(file1)
    f2 = ROOT.TFile.Open(file2)

    if not f1 or not f2:
        print("Error: Could not open one or both files.")
        return False

    # Retrieve histograms
    h1 = f1.Get(hist_name1)
    h2 = f2.Get(hist_name2)

    if not h1 or not h2:
        print("Error: Could not retrieve one or both histograms.")
        return False

    # Check metadata (title, axis labels)
    if h1.GetTitle() != h2.GetTitle():
        print("Mismatch in titles.")
        return False

    if h1.GetXaxis().GetTitle() != h2.GetXaxis().GetTitle():
        print("Mismatch in X-axis titles.")
        return False

    if h1.GetYaxis().GetTitle() != h2.GetYaxis().GetTitle():
        print("Mismatch in Y-axis titles.")
        return False

    # Check bin contents and errors
    if h1.GetNbinsX() != h2.GetNbinsX():
        print("Mismatch in number of bins.")
        return False

    for i in range(1, h1.GetNbinsX() + 1):  # Bin indexing starts at 1 in ROOT
        if h1.GetBinContent(i) != h2.GetBinContent(i):
            print(f"Mismatch in bin content at bin {i}: {h1.GetBinContent(i)} vs {h2.GetBinContent(i)}")
            return False
        if h1.GetBinError(i) != h2.GetBinError(i):
            print(f"Mismatch in bin error at bin {i}: {h1.GetBinError(i)} vs {h2.GetBinError(i)}")
            return False

    # Check underflow and overflow bins
    if h1.GetBinContent(0) != h2.GetBinContent(0) or h1.GetBinContent(h1.GetNbinsX() + 1) != h2.GetBinContent(h2.GetNbinsX() + 1):
        print("Mismatch in underflow or overflow bins.")
        return False

    # Close the files
    f1.Close()
    f2.Close()

    print("Histograms are identical.")
    return True

# Example usage
file1 = "test_dmitry_template.root"
hist_name1 = "test"
file2 = "example_histogram.root"
hist_name2 = "test"

are_identical = compare_histograms(file1, hist_name1, file2, hist_name2)
print(f"Are the histograms identical? {are_identical}")

Histograms are identical.
Are the histograms identical? True


IncrementalExecutor::executeFunction: symbol '_ZN5cling7runtime6gClingE' unresolved while linking [cling interface function]!
You are probably missing the definition of cling::runtime::gCling
Maybe you need to load the corresponding shared library?
IncrementalExecutor::executeFunction: symbol '_ZN5cling7runtime6gClingE' unresolved while linking [cling interface function]!
You are probably missing the definition of cling::runtime::gCling
Maybe you need to load the corresponding shared library?
IncrementalExecutor::executeFunction: symbol '_ZN5cling7runtime6gClingE' unresolved while linking [cling interface function]!
You are probably missing the definition of cling::runtime::gCling
Maybe you need to load the corresponding shared library?
IncrementalExecutor::executeFunction: symbol '_ZN5cling7runtime6gClingE' unresolved while linking [cling interface function]!
You are probably missing the definition of cling::runtime::gCling
Maybe you need to load the corresponding shared library?
Incr